In [3]:
"""
Lightweight Intelligent Intrusion Detection System (LIIDS) for Wireless Sensor Networks
Using Deep Autoencoders with Grid Search Hyperparameter Tuning

Implementation of MSc thesis framework by Salisu Gaya, ZUBAIRU
Department of Electronics and Telecommunications Engineering
Ahmadu Bello University, Zaria

This implementation includes:
1. Data preprocessing for NSL-KDD and UNSW-NB15 datasets
2. Deep autoencoder model architecture with Grid Search hyperparameter optimization
3. Training with resource optimization (early stopping, bottleneck compression)
4. Evaluation metrics (accuracy, recall, false alarm rate)
5. Energy and bandwidth efficiency analysis
"""

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import time
import os
import requests
from itertools import product
import multiprocessing
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

# Define color scheme for plots
COLORS = {
    'primary': '#003366',
    'secondary': '#0066cc',
    'accent': '#ff9900',
    'light_bg': '#f5f5f5',
    'text': '#333333'
}

class LIIDS:
    """
    Lightweight Intelligent Intrusion Detection System (LIIDS) for WSNs
    """

    def __init__(self, input_dim=41, hidden_layers=[128, 64, 32],
                 activation='relu', output_activation='sigmoid',
                 dropout_rate=0.2, learning_rate=0.01,
                 batch_size=32, epochs=50, l1_reg=0.01):
        """
        Initialize the LIIDS model with hyperparameters

        Args:
            input_dim: Number of input features
            hidden_layers: List of neurons in each hidden layer
            activation: Activation function for hidden layers
            output_activation: Activation function for output layer
            dropout_rate: Dropout rate for regularization
            learning_rate: Learning rate for the optimizer
            batch_size: Batch size for training
            epochs: Maximum number of epochs for training
            l1_reg: L1 regularization coefficient
        """
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.output_activation = output_activation
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.l1_reg = l1_reg
        self.threshold = None
        self.original_data_size = input_dim
        self.compressed_data_size = hidden_layers[-1]  # Bottleneck size
        self.model = None
        self.encoder = None
        self.history = None
        self.training_time = 0
        self.scaler = MinMaxScaler()

        # Build the model
        self._build_model()

    def _build_model(self):
        """
        Build the deep autoencoder architecture
        - Encoder: Input → Hidden Layers
        - Decoder: Hidden Layers (reversed) → Output
        - Activations: ReLU for hidden layers, Sigmoid for output
        - Regularization: L1 and Dropout
        """
        # Input layer
        input_layer = Input(shape=(self.input_dim,), name='input')

        # Encoder layers
        encoded = input_layer
        for i, units in enumerate(self.hidden_layers):
            encoded = Dense(units=units,
                           activation=self.activation,
                           kernel_regularizer=l1(self.l1_reg),
                           name=f'encoder_{i+1}')(encoded)
            if i < len(self.hidden_layers) - 1:  # No dropout at bottleneck
                encoded = Dropout(self.dropout_rate)(encoded)

        # Bottleneck layer (last hidden layer)
        bottleneck = encoded

        # Decoder layers (reverse of encoder)
        decoded = bottleneck
        for i, units in enumerate(reversed(self.hidden_layers[:-1])):
            decoded = Dense(units=units,
                           activation=self.activation,
                           kernel_regularizer=l1(self.l1_reg),
                           name=f'decoder_{i+1}')(decoded)
            decoded = Dropout(self.dropout_rate)(decoded)

        # Output layer
        output_layer = Dense(units=self.input_dim,
                            activation=self.output_activation,
                            name='output')(decoded)

        # Create the autoencoder model
        self.model = Model(inputs=input_layer, outputs=output_layer)

        # Create the encoder model (for bottleneck extraction)
        self.encoder = Model(inputs=input_layer, outputs=bottleneck)

        # Compile the model
        self.model.compile(optimizer=Adam(learning_rate=self.learning_rate),
                         loss='mse')

    def preprocess_data(self, X, y=None, fit=True):
        """
        Preprocess the data (normalization/scaling)
        """
        # Scale features to [0,1] range
        if fit:
            X_scaled = self.scaler.fit_transform(X)
        else:
            X_scaled = self.scaler.transform(X)

        return X_scaled, y

    def train(self, X_train, validation_split=0.2, patience=10):
        """
        Train the autoencoder model on normal data only
        """
        print(f"Training model with {self.hidden_layers} neurons, lr={self.learning_rate}, dropout={self.dropout_rate}, L1={self.l1_reg}, batch_size={self.batch_size}")

        # Define callbacks for early stopping
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=patience,
            verbose=0,
            restore_best_weights=True
        )

        # Start timing
        start_time = time.time()

        # Train the model
        self.history = self.model.fit(
            X_train, X_train,  # Autoencoder learns to reconstruct input
            epochs=self.epochs,
            batch_size=self.batch_size,
            validation_split=validation_split,
            callbacks=[early_stopping],
            verbose=0  # Reduced verbosity for grid search
        )

        # Calculate training time
        self.training_time = time.time() - start_time
        print(f"Training completed in {self.training_time:.2f} seconds")

        return self.history

    def set_threshold(self, X_normal, percentile=95):
        """
        Set the threshold for anomaly detection based on reconstruction error of normal data
        """
        # Get reconstruction errors on normal data
        predictions = self.model.predict(X_normal, verbose=0)
        mse = np.mean(np.power(X_normal - predictions, 2), axis=1)

        # Set threshold as the 95th percentile of errors
        self.threshold = np.percentile(mse, percentile)
        print(f"Anomaly threshold set at {self.threshold:.6f} ({percentile}th percentile)")

        return self.threshold

    def predict(self, X):
        """
        Predict anomalies based on reconstruction error
        """
        if self.threshold is None:
            raise ValueError("Threshold not set. Run set_threshold() first.")

        # Calculate reconstruction error
        predictions = self.model.predict(X, verbose=0)
        mse = np.mean(np.power(X - predictions, 2), axis=1)

        # Classify based on threshold
        y_pred = (mse > self.threshold).astype(int)

        return y_pred, mse

    def evaluate(self, X, y_true):
        """
        Evaluate the model performance
        """
        # Get predictions
        y_pred, mse = self.predict(X)

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

        # Calculate false alarm rate
        far = fp / (fp + tn) if (fp + tn) > 0 else 0

        # Calculate ROC curve and AUC
        fpr, tpr, _ = roc_curve(y_true, mse)
        roc_auc = auc(fpr, tpr)

        # Return metrics
        metrics = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'far': far,
            'auc': roc_auc,
            'confusion_matrix': {
                'tn': tn,
                'fp': fp,
                'fn': fn,
                'tp': tp
            },
            'roc': {
                'fpr': fpr,
                'tpr': tpr
            }
        }

        return metrics, mse

    def calculate_resource_efficiency(self):
        """
        Calculate bandwidth and energy efficiency
        """
        # Calculate bandwidth efficiency (Eq. 3.9 in thesis)
        bandwidth_efficiency = ((self.original_data_size - self.compressed_data_size) /
                              self.original_data_size) * 100

        # Calculate energy efficiency (simplified model)
        energy_efficiency = bandwidth_efficiency

        # Return efficiency metrics
        efficiency = {
            'original_size': self.original_data_size,
            'compressed_size': self.compressed_data_size,
            'bandwidth_efficiency': bandwidth_efficiency,
            'energy_efficiency': energy_efficiency,
            'data_reduction_ratio': self.original_data_size/self.compressed_data_size,
            'training_time': self.training_time
        }

        return efficiency

    def plot_training_history(self):
        """Plot training and validation loss over epochs"""
        plt.figure(figsize=(10, 6))
        plt.plot(self.history.history['loss'], label='Training Loss', color=COLORS['primary'])
        plt.plot(self.history.history['val_loss'], label='Validation Loss', color=COLORS['accent'])
        plt.title('Training and Validation Loss', fontsize=14, fontweight='bold')
        plt.xlabel('Epochs', fontsize=12)
        plt.ylabel('Loss (MSE)', fontsize=12)
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

    def plot_confusion_matrix(self, metrics):
        """Plot confusion matrix"""
        cm = np.array([
            [metrics['confusion_matrix']['tn'], metrics['confusion_matrix']['fp']],
            [metrics['confusion_matrix']['fn'], metrics['confusion_matrix']['tp']]
        ])

        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                   xticklabels=['Normal', 'Anomaly'],
                   yticklabels=['Normal', 'Anomaly'])
        plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.show()

    def save_model(self, path="liids_model.h5"):
        """Save the trained model"""
        self.model.save(path)
        print(f"Model saved to {path}")


class GridSearchLIIDS:
    """
    Grid Search for hyperparameter tuning of LIIDS model
    """

    def __init__(self, input_dim, param_grid=None, n_folds=5):
        """
        Initialize the grid search with parameter grid

        Args:
            input_dim: Number of input features
            param_grid: Dictionary of hyperparameters to tune
            n_folds: Number of cross-validation folds
        """
        self.input_dim = input_dim
        self.n_folds = n_folds
        self.best_model = None
        self.best_params = None
        self.best_metrics = None
        self.results = []

        # Default parameter grid if none provided
        if param_grid is None:
            self.param_grid = {
                'hidden_layers': [(128, 64, 32), (256, 128, 64), (64, 32, 16)],
                'learning_rate': [0.001, 0.005, 0.01],
                'batch_size': [32, 64, 128],
                'dropout_rate': [0.1, 0.2, 0.3],
                'l1_reg': [0.001, 0.005, 0.01]
            }
        else:
            self.param_grid = param_grid

    def _evaluate_model(self, X_train, y_train, X_test, y_test, params):
        """
        Evaluate a single model with given parameters
        """
        # Create and train model
        model = LIIDS(
            input_dim=self.input_dim,
            hidden_layers=list(params['hidden_layers']),
            dropout_rate=params['dropout_rate'],
            learning_rate=params['learning_rate'],
            batch_size=params['batch_size'],
            l1_reg=params['l1_reg']
        )

        # Extract normal samples for training
        X_train_normal = X_train[y_train == 0]

        # Preprocess data
        X_train_normal_scaled, _ = model.preprocess_data(X_train_normal)
        X_test_scaled, y_test_scaled = model.preprocess_data(X_test, y_test, fit=False)

        # Train model
        model.train(X_train_normal_scaled, validation_split=0.2, patience=10)

        # Set threshold based on normal data
        model.set_threshold(X_train_normal_scaled)

        # Evaluate model
        metrics, _ = model.evaluate(X_test_scaled, y_test_scaled)

        # Add params to metrics
        metrics.update(params)

        return metrics, model

    def fit(self, X_train, y_train, X_test, y_test, n_jobs=None):
        """
        Perform grid search to find the best hyperparameters

        Args:
            X_train: Training data
            y_train: Training labels
            X_test: Test data
            y_test: Test labels
            n_jobs: Number of parallel jobs (None = use all cores)
        """
        # Generate all parameter combinations
        param_combinations = list(dict(zip(self.param_grid.keys(), values))
                               for values in product(*self.param_grid.values()))

        print(f"Grid searching {len(param_combinations)} hyperparameter combinations...")

        # Function to evaluate one parameter combination
        def evaluate_combination(params):
            try:
                metrics, model = self._evaluate_model(X_train, y_train, X_test, y_test, params)
                return metrics, model, params
            except Exception as e:
                print(f"Error with params {params}: {e}")
                return None, None, params

        # Set number of parallel jobs
        if n_jobs is None:
            n_jobs = max(1, multiprocessing.cpu_count() - 1)

        # Perform grid search in parallel
        results = Parallel(n_jobs=n_jobs)(
            delayed(evaluate_combination)(params) for params in param_combinations
        )

        # Filter out None results (errors)
        valid_results = [r for r in results if r[0] is not None]

        # Sort by composite score (accuracy + recall - far)
        sorted_results = sorted(
            valid_results,
            key=lambda x: (x[0]['accuracy'] + x[0]['recall'] - x[0]['far']),
            reverse=True
        )

        if not sorted_results:
            raise ValueError("No valid parameter combinations found")

        # Get best results
        best_metrics, best_model, best_params = sorted_results[0]

        self.best_metrics = best_metrics
        self.best_model = best_model
        self.best_params = best_params
        self.results = sorted_results

        print("\nGrid Search complete!")
        print(f"Best parameters: {best_params}")
        print(f"Best performance: Accuracy={best_metrics['accuracy']:.4f}, "
              f"Recall={best_metrics['recall']:.4f}, "
              f"FAR={best_metrics['far']:.4f}")

        return self.best_model, self.best_params, self.best_metrics

    def plot_grid_search_results(self):
        """Plot grid search results"""
        # Extract results for plotting
        results_df = pd.DataFrame([r[0] for r in self.results])

        # Plot accuracy vs. different hyperparameters
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # Learning rate vs accuracy
        sns.boxplot(x='learning_rate', y='accuracy', data=results_df, ax=axes[0, 0])
        axes[0, 0].set_title('Learning Rate vs Accuracy')
        axes[0, 0].grid(True, linestyle='--', alpha=0.7)

        # Dropout rate vs accuracy
        sns.boxplot(x='dropout_rate', y='accuracy', data=results_df, ax=axes[0, 1])
        axes[0, 1].set_title('Dropout Rate vs Accuracy')
        axes[0, 1].grid(True, linestyle='--', alpha=0.7)

        # L1 regularization vs accuracy
        sns.boxplot(x='l1_reg', y='accuracy', data=results_df, ax=axes[1, 0])
        axes[1, 0].set_title('L1 Regularization vs Accuracy')
        axes[1, 0].grid(True, linestyle='--', alpha=0.7)

        # Batch size vs accuracy
        sns.boxplot(x='batch_size', y='accuracy', data=results_df, ax=axes[1, 1])
        axes[1, 1].set_title('Batch Size vs Accuracy')
        axes[1, 1].grid(True, linestyle='--', alpha=0.7)

        plt.tight_layout()
        plt.show()

        # Plot metrics for different network architectures
        plt.figure(figsize=(12, 6))

        architecture_groups = results_df.groupby('hidden_layers').agg({
            'accuracy': 'mean',
            'recall': 'mean',
            'precision': 'mean',
            'far': 'mean'
        }).reset_index()

        architectures = [str(tuple(eval(arch))) for arch in architecture_groups['hidden_layers']]

        x = np.arange(len(architectures))
        width = 0.2

        plt.bar(x - width*1.5, architecture_groups['accuracy'], width, label='Accuracy', color=COLORS['primary'])
        plt.bar(x - width/2, architecture_groups['recall'], width, label='Recall', color=COLORS['secondary'])
        plt.bar(x + width/2, architecture_groups['precision'], width, label='Precision', color=COLORS['accent'])
        plt.bar(x + width*1.5, architecture_groups['far'], width, label='FAR', color='red')

        plt.xlabel('Network Architecture (hidden layers)')
        plt.ylabel('Metric Value')
        plt.title('Performance Metrics by Network Architecture')
        plt.xticks(x, architectures)
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()


class DataProcessor:
    """
    Class for handling data loading and preprocessing
    """

    @staticmethod
    def load_nsl_kdd(train_path=None, test_path=None):
        """
        Load and preprocess the NSL-KDD dataset
        """
        print("\nLoading NSL-KDD dataset...")

        # Define column names
        columns = [
            'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
            'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
            'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
            'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
            'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
            'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
            'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
            'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
            'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
            'dst_host_srv_rerror_rate', 'label', 'difficulty'
        ]

        # If paths are not provided, download the dataset
        if train_path is None or test_path is None:
            train_path = 'NSL-KDD/KDDTrain+.txt'
            test_path = 'NSL-KDD/KDDTest+.txt'

            # Create directory if it doesn't exist
            if not os.path.exists('NSL-KDD'):
                os.makedirs('NSL-KDD')

            # Download files if they don't exist
            if not os.path.exists(train_path):
                print("Downloading NSL-KDD training set...")
                train_url = 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain%2B.txt'
                response = requests.get(train_url)
                with open(train_path, 'wb') as f:
                    f.write(response.content)

            if not os.path.exists(test_path):
                print("Downloading NSL-KDD test set...")
                test_url = 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest%2B.txt'
                response = requests.get(test_url)
                with open(test_path, 'wb') as f:
                    f.write(response.content)

        # Load data
        train_data = pd.read_csv(train_path, header=None, names=columns)
        test_data = pd.read_csv(test_path, header=None, names=columns)

        # Display dataset information
        print(f"NSL-KDD Training set shape: {train_data.shape}")
        print(f"NSL-KDD Test set shape: {test_data.shape}")

        # Combine train and test for preprocessing
        data = pd.concat([train_data, test_data], axis=0)

        # Convert labels to binary (normal vs attack)
        data['binary_label'] = data['label'].apply(lambda x: 0 if x == 'normal' else 1)

        # One-hot encode categorical features
        categorical_cols = ['protocol_type', 'service', 'flag']
        data = pd.get_dummies(data, columns=categorical_cols, drop_first=False)

        # Drop unnecessary columns
        data = data.drop(['label', 'difficulty'], axis=1)

        # Extract features and labels
        X = data.drop('binary_label', axis=1)
        y = data['binary_label'].values

        # Split back into train and test (maintain original split)
        train_size = train_data.shape[0]
        X_train = X.iloc[:train_size]
        y_train = y[:train_size]
        X_test = X.iloc[train_size:]
        y_test = y[train_size:]

        print(f"Features shape after preprocessing: {X.shape}")
        print(f"Number of normal samples: {(y == 0).sum()}")
        print(f"Number of attack samples: {(y == 1).sum()}")
        print(f"Class distribution in training set: {np.bincount(y_train)}")
        print(f"Class distribution in test set: {np.bincount(y_test)}")

        return X_train, y_train, X_test, y_test

    @staticmethod
    def load_unsw_nb15(train_path=None, test_path=None):
        """
        Load and preprocess the UNSW-NB15 dataset
        """
        print("\nLoading UNSW-NB15 dataset...")

        try:
            # If paths are not provided, download the dataset
            if train_path is None or test_path is None:
                train_path = 'UNSW-NB15/UNSW-NB15_TRAIN.csv'
                test_path = 'UNSW-NB15/UNSW-NB15_TEST.csv'

                # Create directory if it doesn't exist
                if not os.path.exists('UNSW-NB15'):
                    os.makedirs('UNSW-NB15')

                # Try multiple sources for downloading
                sources = [
                    ('https://research.unsw.edu.au/sites/default/files/documents/UNSW_NB15_TRAIN.csv',
                     'https://research.unsw.edu.au/sites/default/files/documents/UNSW_NB15_TEST.csv'),
                    ('https://raw.githubusercontent.com/defcom17/NSL_KDD/master/UNSW-NB15_TRAIN.csv',
                     'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/UNSW-NB15_TEST.csv')
                ]

                for train_url, test_url in sources:
                    try:
                        # Download training set
                        if not os.path.exists(train_path):
                            print(f"Trying to download from {train_url}...")
                            response = requests.get(train_url, timeout=10)
                            response.raise_for_status()
                            with open(train_path, 'wb') as f:
                                f.write(response.content)

                        # Download test set
                        if not os.path.exists(test_path):
                            print(f"Trying to download from {test_url}...")
                            response = requests.get(test_url, timeout=10)
                            response.raise_for_status()
                            with open(test_path, 'wb') as f:
                                f.write(response.content)

                        # If both files exist, break the loop
                        if os.path.exists(train_path) and os.path.exists(test_path):
                            break

                    except Exception as e:
                        print(f"Download failed: {e}")
                        continue

            # Load data
            train_data = pd.read_csv(train_path, low_memory=False)
            test_data = pd.read_csv(test_path, low_memory=False)

            # Verify data format
            if 'label' not in train_data.columns or 'label' not in test_data.columns:
                raise ValueError("Dataset is missing 'label' column")

            # Display dataset information
            print(f"UNSW-NB15 Training set shape: {train_data.shape}")
            print(f"UNSW-NB15 Test set shape: {test_data.shape}")

            # Combine train and test for preprocessing
            data = pd.concat([train_data, test_data], axis=0)

            # Drop irrelevant columns
            if 'id' in data.columns:
                data = data.drop(['id'], axis=1)
            if 'attack_cat' in data.columns:
                data = data.drop(['attack_cat'], axis=1)

            # Handle missing values
            numeric_cols = data.select_dtypes(include=['number']).columns
            for col in numeric_cols:
                if data[col].isnull().sum() > 0:
                    data[col] = data[col].fillna(data[col].median())

            categorical_cols = data.select_dtypes(include=['object']).columns
            for col in categorical_cols:
                if data[col].isnull().sum() > 0:
                    data[col] = data[col].fillna(data[col].mode()[0])

            # One-hot encode categorical features
            categorical_cols = [col for col in ['proto', 'service', 'state'] if col in data.columns]
            data = pd.get_dummies(data, columns=categorical_cols, drop_first=False)

            # Extract features and labels
            X = data.drop('label', axis=1)
            y = data['label'].values

            # Split back into train and test (maintain original split)
            train_size = train_data.shape[0]
            X_train = X.iloc[:train_size]
            y_train = y[:train_size]
            X_test = X.iloc[train_size:]
            y_test = y[train_size:]

            print(f"Features shape after preprocessing: {X.shape}")
            print(f"Number of normal samples: {(y == 0).sum()}")
            print(f"Number of attack samples: {(y == 1).sum()}")
            print(f"Class distribution in training set: {np.bincount(y_train)}")
            print(f"Class distribution in test set: {np.bincount(y_test)}")

            return X_train, y_train, X_test, y_test

        except Exception as e:
            print(f"Error loading UNSW-NB15 dataset: {e}")
            print("Please download the UNSW-NB15 dataset manually and place in UNSW-NB15/ directory")
            return None, None, None, None


def run_nsl_kdd_experiment(use_grid_search=True):
    """Run experiment on NSL-KDD dataset with grid search"""
    print("\n" + "="*80)
    print("NSL-KDD DATASET EXPERIMENT WITH GRID SEARCH")
    print("="*80)

    # Load NSL-KDD dataset
    X_train, y_train, X_test, y_test = DataProcessor.load_nsl_kdd()

    if use_grid_search:
        # Define parameter grid
        param_grid = {
            'hidden_layers': [(128, 64, 32), (256, 128, 64), (64, 32, 16)],
            'learning_rate': [0.001, 0.005, 0.01],
            'batch_size': [32, 64, 128],
            'dropout_rate': [0.1, 0.2, 0.3],
            'l1_reg': [0.001, 0.005, 0.01]
        }

        # Perform grid search
        grid_search = GridSearchLIIDS(input_dim=X_train.shape[1], param_grid=param_grid)
        best_model, best_params, best_metrics = grid_search.fit(X_train, y_train, X_test, y_test)

        # Plot grid search results
        grid_search.plot_grid_search_results()

        # Print best hyperparameters
        print("\nBest Hyperparameters:")
        for param, value in best_params.items():
            print(f"{param}: {value}")

        # Print best metrics
        print("\nBest Model Performance:")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'far', 'auc']:
            print(f"{metric}: {best_metrics[metric]:.4f}")

        # Save best model
        best_model.save_model("best_nsl_kdd_model.h5")

        return best_model, best_metrics, best_params

    else:
        # Use default hyperparameters without grid search
        print("\nRunning with default hyperparameters (no grid search)")

        # Extract normal samples for training the autoencoder
        X_train_normal = X_train[y_train == 0]

        # Initialize and train LIIDS
        liids = LIIDS(input_dim=X_train.shape[1])

        # Preprocess data
        X_train_normal_scaled, _ = liids.preprocess_data(X_train_normal)
        X_test_scaled, y_test_scaled = liids.preprocess_data(X_test, y_test, fit=False)

        # Train the model
        liids.train(X_train_normal_scaled, validation_split=0.2, patience=10)

        # Set threshold based on normal data
        liids.set_threshold(X_train_normal_scaled)

        # Evaluate model
        metrics, _ = liids.evaluate(X_test_scaled, y_test_scaled)

        # Print metrics
        print("\nDefault Model Performance:")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'far', 'auc']:
            print(f"{metric}: {metrics[metric]:.4f}")

        # Save model
        liids.save_model("default_nsl_kdd_model.h5")

        return liids, metrics, None


def run_unsw_nb15_experiment(use_grid_search=True):
    """Run experiment on UNSW-NB15 dataset with grid search"""
    print("\n" + "="*80)
    print("UNSW-NB15 DATASET EXPERIMENT WITH GRID SEARCH")
    print("="*80)

    # Load UNSW-NB15 dataset
    X_train, y_train, X_test, y_test = DataProcessor.load_unsw_nb15()

    # Check if dataset loaded successfully
    if X_train is None:
        print("Skipping UNSW-NB15 experiment due to dataset loading failure")
        return None, None, None

    if use_grid_search:
        # Define parameter grid
        param_grid = {
            'hidden_layers': [(128, 64, 32), (256, 128, 64), (64, 32, 16)],
            'learning_rate': [0.001, 0.005, 0.01],
            'batch_size': [32, 64, 128],
            'dropout_rate': [0.1, 0.2, 0.3],
            'l1_reg': [0.001, 0.005, 0.01]
        }

        # Perform grid search
        grid_search = GridSearchLIIDS(input_dim=X_train.shape[1], param_grid=param_grid)
        best_model, best_params, best_metrics = grid_search.fit(X_train, y_train, X_test, y_test)

        # Plot grid search results
        grid_search.plot_grid_search_results()

        # Print best hyperparameters
        print("\nBest Hyperparameters:")
        for param, value in best_params.items():
            print(f"{param}: {value}")

        # Print best metrics
        print("\nBest Model Performance:")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'far', 'auc']:
            print(f"{metric}: {best_metrics[metric]:.4f}")

        # Save best model
        best_model.save_model("best_unsw_nb15_model.h5")

        return best_model, best_metrics, best_params

    else:
        # Use default hyperparameters without grid search
        print("\nRunning with default hyperparameters (no grid search)")

        # Extract normal samples for training the autoencoder
        X_train_normal = X_train[y_train == 0]

        # Initialize and train LIIDS
        liids = LIIDS(input_dim=X_train.shape[1])

        # Preprocess data
        X_train_normal_scaled, _ = liids.preprocess_data(X_train_normal)
        X_test_scaled, y_test_scaled = liids.preprocess_data(X_test, y_test, fit=False)

        # Train the model
        liids.train(X_train_normal_scaled, validation_split=0.2, patience=10)

        # Set threshold based on normal data
        liids.set_threshold(X_train_normal_scaled)

        # Evaluate model
        metrics, _ = liids.evaluate(X_test_scaled, y_test_scaled)

        # Print metrics
        print("\nDefault Model Performance:")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'far', 'auc']:
            print(f"{metric}: {metrics[metric]:.4f}")

        # Save model
        liids.save_model("default_unsw_nb15_model.h5")

        return liids, metrics, None


def compare_results(nsl_kdd_results, unsw_nb15_results):
    """Compare results from different datasets"""
    print("\n" + "="*80)
    print("RESULTS COMPARISON")
    print("="*80)

    # Unpack results
    nsl_kdd_model, nsl_kdd_metrics, nsl_kdd_params = nsl_kdd_results

    if unsw_nb15_results[0] is None:
        print("Cannot compare results because UNSW-NB15 experiment failed")

        # Create comparison table with just NSL-KDD
        results = pd.DataFrame({
            'NSL-KDD': [
                nsl_kdd_metrics['accuracy'],
                nsl_kdd_metrics['precision'],
                nsl_kdd_metrics['recall'],
                nsl_kdd_metrics['f1'],
                nsl_kdd_metrics['far'],
                nsl_kdd_metrics['auc']
            ]
        }, index=['Accuracy', 'Precision', 'Recall (DR)', 'F1-Score', 'False Alarm Rate', 'AUC'])

        # Format percentages
        results = results.applymap(lambda x: f"{x:.4f}")

        print("\nPerformance Metrics:")
        print(results)

        return results

    unsw_nb15_model, unsw_nb15_metrics, unsw_nb15_params = unsw_nb15_results

    # Create comparison table
    results = pd.DataFrame({
        'NSL-KDD': [
            nsl_kdd_metrics['accuracy'],
            nsl_kdd_metrics['precision'],
            nsl_kdd_metrics['recall'],
            nsl_kdd_metrics['f1'],
            nsl_kdd_metrics['far'],
            nsl_kdd_metrics['auc']
        ],
        'UNSW-NB15': [
            unsw_nb15_metrics['accuracy'],
            unsw_nb15_metrics['precision'],
            unsw_nb15_metrics['recall'],
            unsw_nb15_metrics['f1'],
            unsw_nb15_metrics['far'],
            unsw_nb15_metrics['auc']
        ]
    }, index=['Accuracy', 'Precision', 'Recall (DR)', 'F1-Score', 'False Alarm Rate', 'AUC'])

    # Format percentages
    results = results.applymap(lambda x: f"{x:.4f}")

    print("\nPerformance Metrics Comparison:")
    print(results)

    # Compare with results from thesis
    thesis_results = pd.DataFrame({
        'NSL-KDD (Thesis)': ['0.9976', '0.9935', '0.0065'],
        'UNSW-NB15 (Thesis)': ['0.9852', '0.9873', '0.0127'],
        'NSL-KDD (Grid Search)': [
            f"{nsl_kdd_metrics['accuracy']:.4f}",
            f"{nsl_kdd_metrics['recall']:.4f}",
            f"{nsl_kdd_metrics['far']:.4f}"
        ],
        'UNSW-NB15 (Grid Search)': [
            f"{unsw_nb15_metrics['accuracy']:.4f}",
            f"{unsw_nb15_metrics['recall']:.4f}",
            f"{unsw_nb15_metrics['far']:.4f}"
        ]
    }, index=['Accuracy', 'Recall (DR)', 'False Alarm Rate'])

    print("\nComparison with Thesis Results:")
    print(thesis_results)

    # Plot comparison
    metrics = ['Accuracy', 'Recall (DR)', 'False Alarm Rate']
    datasets = ['NSL-KDD', 'UNSW-NB15']

    # Create figure with multiple subplots
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    for i, metric in enumerate(metrics):
        ax = axes[i]

        # Get values
        thesis_values = [float(thesis_results.loc[metric, f'{ds} (Thesis)']) for ds in datasets]
        grid_search_values = [float(thesis_results.loc[metric, f'{ds} (Grid Search)'].replace(',', '.'))
                              for ds in datasets]

        # Create grouped bar chart
        x = np.arange(len(datasets))
        width = 0.35

        ax.bar(x - width/2, thesis_values, width, label='Thesis Results', color=COLORS['primary'])
        ax.bar(x + width/2, grid_search_values, width, label='Grid Search Results', color=COLORS['accent'])

        # Add labels and title
        ax.set_title(metric, fontsize=14, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(datasets)
        ax.set_ylim(0, 1.0 if metric != 'False Alarm Rate' else 0.1)

        # Add value labels
        for j, v in enumerate(thesis_values):
            ax.text(j - width/2, v + 0.01, f"{v:.4f}", ha='center', fontsize=10)

        for j, v in enumerate(grid_search_values):
            ax.text(j + width/2, v + 0.01, f"{v:.4f}", ha='center', fontsize=10)

        # Add legend and grid
        if i == 1:
            ax.legend()
        ax.grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.savefig('results_comparison.png')
    plt.show()

    # Return results dataframe
    return results


def main():
    """Main function to run all experiments"""
    print("\n" + "="*80)
    print("LIGHTWEIGHT INTELLIGENT INTRUSION DETECTION SYSTEM (LIIDS)")
    print("FOR WIRELESS SENSOR NETWORKS USING DEEP AUTOENCODERS")
    print("WITH GRID SEARCH HYPERPARAMETER TUNING")
    print("="*80)

    # Run NSL-KDD experiment with grid search
    nsl_kdd_results = run_nsl_kdd_experiment(use_grid_search=True)

    # Run UNSW-NB15 experiment with grid search
    unsw_nb15_results = run_unsw_nb15_experiment(use_grid_search=True)

    # Compare results
    compare_results(nsl_kdd_results, unsw_nb15_results)

    print("\n" + "="*80)
    print("GRID SEARCH EXPERIMENTS COMPLETED SUCCESSFULLY")
    print("="*80)


if __name__ == "__main__":
    main()


LIGHTWEIGHT INTELLIGENT INTRUSION DETECTION SYSTEM (LIIDS)
FOR WIRELESS SENSOR NETWORKS USING DEEP AUTOENCODERS
WITH GRID SEARCH HYPERPARAMETER TUNING

NSL-KDD DATASET EXPERIMENT WITH GRID SEARCH

Loading NSL-KDD dataset...
NSL-KDD Training set shape: (125973, 43)
NSL-KDD Test set shape: (22544, 43)
Features shape after preprocessing: (148517, 122)
Number of normal samples: 77054
Number of attack samples: 71463
Class distribution in training set: [67343 58630]
Class distribution in test set: [ 9711 12833]
Grid searching 243 hyperparameter combinations...


KeyboardInterrupt: 